In [21]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet50
from PIL import Image
import pandas as pd
import cv2

In [22]:
#model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)
model.eval()

Using cache found in C:\Users\andre/.cache\torch\hub\pytorch_vision_v0.10.0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [35]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to classify an image
def classify_image(image):
    input_tensor = preprocess(image)
    input_tensor = input_tensor.unsqueeze(0)  # create a mini-batch
    with torch.no_grad():
        if torch.cuda.is_available():
            input_tensor = input_tensor.to('cuda')
            model.to('cuda')
        output = model(input_tensor)
    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    top_prob, top_catid = torch.topk(probabilities, 1)
    return top_catid.item(), top_prob.item()

In [36]:
# Function to extract the 5th frame from a video
def extract_fifth_frame(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return None
    frame_count = 0
    while frame_count < 5:
        ret, frame = cap.read()
        frame_count += 1
    cap.release()
    if ret:
        frame_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        return frame_image
    return None

In [37]:
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

In [38]:
# Path to the folder containing video files
video_folder = r"C:\Users\andre\OneDrive\Documents\GitHub\Brainvision_Project\Motion_files\videos_processed"

# Create empty lists to store results
file_names = []
predicted_categories = []
probabilities = []

# Process each video file in the folder
for video_file in os.listdir(video_folder):
    if video_file.endswith(".mp4"):  # Adjust the file extension as needed
        video_path = os.path.join(video_folder, video_file)
        frame_image = extract_first_frame(video_path)
        if frame_image is not None:
            predicted_category, probability = classify_image(frame_image)
            file_names.append(video_file)
            predicted_categories.append(categories[predicted_category])
            probabilities.append(probability)

# Create a DataFrame to store results
results_df = pd.DataFrame({'File': file_names, 'Predicted_Category': predicted_categories, 'Probability': probabilities})

# Save the results DataFrame to a CSV file
results_df.to_csv('object_labels.csv', index=False)